In [ ]:
from pyspark.sql import SparkSession
from bronze_properties import BronzeProcessor

spark = SparkSession.builder.getOrCreate()


data_bucket = spark.conf.get("spark.databricks.bundle.variables.data_bucket")
gcs_prefix = spark.conf.get("spark.databricks.bundle.variables.gcs_prefix")
gcs_path = f"gs://{data_bucket}/{gcs_prefix}"

# Initialize BronzeProcessor with dbutils
bronze_processor = BronzeProcessor(spark, gcs_path, None, dbutils)


try:
    import dlt
except ImportError:
    class dlt:
        @staticmethod
        def table(comment=None, table_properties=None):
            def decorator(f): return f
            return decorator

        @staticmethod
        def expect_all(rules):
            def decorator(f): return f
            return decorator


@dlt.table(
    comment="Raw property data in bronze layer (streaming via Auto Loader)",
    table_properties={"quality": "bronze"}
)
def br_properties():
    raw_df = bronze_processor.ingest_raw_property_data()
    return raw_df